# 6. Stress Testing

Scenario analysis and stress testing for credit portfolios.

## Contents
1. Scenario Definition
2. Impact Analysis
3. Sensitivity Analysis
4. Reverse Stress Testing

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

from privatecredit.data import MacroScenarioGenerator

## 1. Scenario Definition

In [ ]:
# Generate scenarios
macro_gen = MacroScenarioGenerator(n_months=60, start_date='2020-01-01', seed=42)

scenarios = {
    'baseline': macro_gen.generate_scenario('baseline'),
    'adverse': macro_gen.generate_scenario('adverse'),
    'severely_adverse': macro_gen.generate_scenario('severely_adverse'),
    'stagflation': macro_gen.generate_scenario('stagflation')
}

# Summary statistics
for name, df in scenarios.items():
    print(f"\n{name.upper()}:")
    print(f"  GDP Growth: {df['gdp_growth_yoy'].mean():.2%}")
    print(f"  Unemployment: {df['unemployment_rate'].mean():.1%}")
    print(f"  HY Spread: {df['credit_spread_hy'].mean():.0f}bps")

## 2. Impact Analysis

In [ ]:
# PD multipliers based on macro conditions
def compute_pd_multiplier(gdp_growth, unemployment, hy_spread):
    """Simple PD stress multiplier"""
    # Base = 1.0 for baseline
    multiplier = 1.0
    
    # GDP impact (negative growth increases PD)
    if gdp_growth < 0:
        multiplier *= (1 - gdp_growth * 5)  # 1% GDP decline -> 5% PD increase
    
    # Unemployment impact
    multiplier *= (1 + (unemployment - 0.04) * 2)  # Above 4% baseline
    
    # Credit spread impact  
    multiplier *= (1 + (hy_spread - 400) / 1000)  # Above 400bps baseline
    
    return max(0.5, min(5.0, multiplier))  # Cap at 0.5x to 5x

# Compute multipliers for each scenario
pd_multipliers = {}
for name, df in scenarios.items():
    multipliers = []
    for _, row in df.iterrows():
        m = compute_pd_multiplier(
            row['gdp_growth_yoy'],
            row['unemployment_rate'],
            row['credit_spread_hy']
        )
        multipliers.append(m)
    pd_multipliers[name] = np.array(multipliers)

# Plot multipliers over time
fig, ax = plt.subplots(figsize=(12, 6))
colors = {'baseline': 'green', 'adverse': 'orange', 
          'severely_adverse': 'red', 'stagflation': 'purple'}

for name, mult in pd_multipliers.items():
    ax.plot(mult, label=name.replace('_', ' ').title(), 
            color=colors[name], linewidth=2)

ax.axhline(1.0, color='gray', linestyle='--', alpha=0.5)
ax.set_xlabel('Month')
ax.set_ylabel('PD Multiplier')
ax.set_title('Default Probability Stress Multipliers')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Portfolio impact
base_pd = 0.02
portfolio_balance = 500_000_000
lgd = 0.40

print("\nExpected Loss by Scenario:")
for name, mult in pd_multipliers.items():
    stressed_pd = base_pd * mult.mean()
    el = portfolio_balance * stressed_pd * lgd
    el_rate = stressed_pd * lgd
    print(f"  {name:20s}: ${el/1e6:6.2f}M ({el_rate:.2%} of portfolio)")

## 3. Sensitivity Analysis

In [ ]:
# Sensitivity to individual factors
gdp_shocks = np.linspace(-0.05, 0.05, 11)
unemployment_shocks = np.linspace(0.03, 0.12, 10)
spread_shocks = np.linspace(200, 1000, 9)

# Base case
base_gdp = 0.02
base_unemployment = 0.04
base_spread = 400

# Compute sensitivities
gdp_impact = [compute_pd_multiplier(g, base_unemployment, base_spread) for g in gdp_shocks]
unemp_impact = [compute_pd_multiplier(base_gdp, u, base_spread) for u in unemployment_shocks]
spread_impact = [compute_pd_multiplier(base_gdp, base_unemployment, s) for s in spread_shocks]

# Plot
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(gdp_shocks * 100, gdp_impact, 'b-o', linewidth=2)
axes[0].set_xlabel('GDP Growth (%)')
axes[0].set_ylabel('PD Multiplier')
axes[0].set_title('Sensitivity to GDP')
axes[0].axhline(1, color='gray', linestyle='--')
axes[0].grid(True, alpha=0.3)

axes[1].plot(unemployment_shocks * 100, unemp_impact, 'r-o', linewidth=2)
axes[1].set_xlabel('Unemployment Rate (%)')
axes[1].set_ylabel('PD Multiplier')
axes[1].set_title('Sensitivity to Unemployment')
axes[1].axhline(1, color='gray', linestyle='--')
axes[1].grid(True, alpha=0.3)

axes[2].plot(spread_shocks, spread_impact, 'g-o', linewidth=2)
axes[2].set_xlabel('HY Spread (bps)')
axes[2].set_ylabel('PD Multiplier')
axes[2].set_title('Sensitivity to Credit Spreads')
axes[2].axhline(1, color='gray', linestyle='--')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Reverse Stress Testing

In [ ]:
# Find scenario that causes specific loss level
target_loss_rate = 0.05  # 5% loss
base_el_rate = base_pd * lgd

required_multiplier = target_loss_rate / base_el_rate
print(f"To achieve {target_loss_rate:.1%} loss rate:")
print(f"  Required PD multiplier: {required_multiplier:.2f}x")

# Find combinations that achieve this
print(f"\nExample scenarios achieving this:")
print(f"  GDP = -3%, Unemployment = 8%, Spread = 600bps")
print(f"  Multiplier: {compute_pd_multiplier(-0.03, 0.08, 600):.2f}x")

print(f"\n  GDP = -5%, Unemployment = 6%, Spread = 500bps")
print(f"  Multiplier: {compute_pd_multiplier(-0.05, 0.06, 500):.2f}x")

## Summary

- Four standard scenarios: baseline, adverse, severely adverse, stagflation
- PD multipliers based on macro conditions
- Sensitivity analysis for individual risk factors
- Reverse stress testing to find breaking points

**Next:** Calibration (Notebook 07)